# Agent Testing Notebook

This notebook is for testing the `LangGraphAgent` with different agent architectures.

## Setup

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Callable, List, Optional, cast, Dict, Literal, Union
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool, tool
from langchain_core.language_models import BaseChatModel

In [2]:
# from src.langgraph.app.core.langgraph.smolagent.smol_agent import SMOLAgent
from src.langgraph.app.core.langgraph.agents import create_agent

In [4]:

import os
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Callable, List, Optional, cast, Dict, Literal, Union
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool, Tool


class SearchToolInput(BaseModel):
    query: str = Field(..., description="The search query to look up.")
    max_results: Optional[int] = Field(default=10, description="The maximum number of search results to return.")

# Define the Tool
class TavilySearchTool:
    def __init__(self, max_results: int = 10):
        self.max_results = max_results

    def search(self, query: str) -> Optional[List[Dict[str, Any]]]:
        """
        Perform a web search using the Tavily search engine.
        """
        try:
            # Initialize the Tavily search tool with the configured max_results
            search_tool = TavilySearchResults(max_results=self.max_results, tavily_api_key=os.getenv("TAVILY_API_KEY"))

            # Perform the search (synchronously)
            result = search_tool.invoke({"query": query})

            # Return the search results
            return result
        except Exception as e:
            return {"error": str(e)}

tavily_search_tool = Tool(
    name="Tavily_Search",
    func=TavilySearchTool().search,
    description="Performs web searches using the Tavily search engine, providing accurate and trusted results for general queries.",
    args_schema=SearchToolInput
)


In [4]:
tool_2 = Tool(
    name="Echo_Tool",
    func=lambda x: f"Echo: {x}",
    description="A tool that echoes the input it receives.",
    args_schema=SearchToolInput
)

tool_3 = Tool(
    name="Echo_Tool_2",
    func=lambda x: f"Echo_2: {x}",
    description="A tool that echoes the input it receives.",
    args_schema=SearchToolInput
)

## 1. Base Agent Test

In [1]:
# from src.langgraph.app.core.langgraph.toolsagent.agents import create_agent
from src.langgraph.app.core.langgraph.agents import create_agent
from src.langgraph.app.core.langgraph.deepagents import DeepResearchAgent 
from src.langgraph.app.core.langgraph.smolagent import SMOLAgent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
import asyncio
import os
import logging
from typing import List, Sequence, TypedDict, Annotated, Optional, Dict, Any

from dotenv import load_dotenv
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.base import BaseCheckpointSaver
from langgraph.graph.message import add_messages
from langgraph.graph.state import CompiledStateGraph
from langgraph.managed import RemainingSteps

# Local application imports
from src.langgraph.app.core.langgraph.agents import create_agent
from src.langgraph.app.core.langgraph.smolagent import SMOLAgent
# from src.langgraph.app.core.langgraph.toolsagent import ToolsAgent
from src.langgraph.app.core.langgraph.deepagents import DeepResearchAgent
from src.langgraph.app.core.langgraph.swarm import SwarmState, create_handoff_tool, create_swarm


llm = ChatOpenAI(model="gpt-4", temperature=0)
selector_llm = ChatOpenAI(model="gpt-4", temperature=0)

memory = MemorySaver()

ImportError: cannot import name 'Client' from 'primp.primp' (c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\primp\primp.pyd)

In [ ]:

from src.langgraph.app.core.langgraph import MORGANA

# Load environment variables from a .env file
load_dotenv()

ImportError: cannot import name 'MORGANA' from 'src.langgraph.app.core.langgraph' (unknown location)

In [10]:
agent = DeepResearchAgent(
    llm=llm,
    checkpointer=memory
)
my_agent = await agent.build()


# 4. Define the research task and run the agent
thread_config = {"configurable": {"thread_id": "deep-research-thread-2"}}
query = "What were the main causes and consequences of the 2008 financial crisis? Write the report in Spanish."

initial_input = {"messages": [HumanMessage(content=query)]}

print(f"--- Running Deep Research Agent for query: '{query}' ---")

# The .invoke() method is synchronous and works here because my_agent is now correctly instantiated.
# Note: Your main() example used astream_events for streaming, which is also a great option.
result = my_agent.invoke(initial_input, config=thread_config)

# The final output structure might vary, adjust this key if needed.
if result.get('messages') and isinstance(result['messages'], list) and len(result['messages']) > 1:
        final_report_content = result['messages'][-1].content
        # Assuming the report is in the content of the last message
        print("\n--- FINAL REPORT ---")
        print(final_report_content)
else:
        print("Could not find final report in the result.")
        print(result)




2025-09-09 14:54:46,747 - src.langgraph.app.core.langgraph.deepagents.deep_research - INFO - DeepResearchAgent factory initialized.
2025-09-09 14:54:46,748 - src.langgraph.app.core.langgraph.deepagents.deep_research - INFO - Building the deep research agent executor...
2025-09-09 14:54:46,748 - src.langgraph.app.core.langgraph.deepagents.deep_research - INFO - Building the deep research agent executor...
c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an obje

--- Running Deep Research Agent for query: 'What were the main causes and consequences of the 2008 financial crisis? Write the report in Spanish.' ---


2025-09-09 14:54:50,570 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:54:52,391 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:54:52,391 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:54:54,164 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:54:54,164 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:55:10,492 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:55:10,492 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:55:12,864 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-09 14:55:12,864 - httpx 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8329 tokens (4367 in the messages, 3962 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}